## Library imports

In [1]:
!pip install timm

     |████████████████████████████████| 244 kB 3.0 MB/s eta 0:00:01


In [2]:
# basic imports
import os
import numpy as np
import pandas as pd
import random
import itertools
from tqdm.notebook import tqdm
import math

# augumentations library
from albumentations.pytorch import ToTensorV2
from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop, HorizontalFlip, VerticalFlip, 
    RandomBrightnessContrast,ShiftScaleRotate, Cutout, CoarseDropout, 
    IAAAdditiveGaussianNoise, Transpose, MotionBlur, MedianBlur, GaussianBlur, HueSaturationValue
    )
import albumentations as A
import cv2

# DL library imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from  torch.cuda.amp import autocast, GradScaler

# timm import
import timm

# metrics calculation
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold, StratifiedKFold

# basic plotting library
import matplotlib.pyplot as plt

# interactive plots
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings  
warnings.filterwarnings('ignore')

## Config params

In [26]:
class CFG:
    # pipeline parameters
    SEED        = 42
    NUM_CLASSES = 5
    TGT_LABEL   = 'label'
    N_FOLDS     = 5 
    VAL_BATCH_SIZE  = 32
    SIZE            = [512,512]
    NUM_WORKERS     = 4

    # model parameters
    WGT_PATH    = '../input/cassava-final-submission-weight-files'
    
    # tf_efficientnet_b4_ns, vit_base_patch16_384, tf_efficientnet_b3_ns, tf_efficientnet_b3_ns
    MODEL_ARCH  = 'resnext50_32x4d'           
    
    # eff_b3_baseline, resnext50_32x4d_baseline, vit_baseline, eff_b4_baseline
    WGT_MODEL   = 'resnext50_baseline_v2'  
    

TRAIN_PATH = '../input/cassava-leaf-disease-classification/train_images'
NPY_FOLDER = '../input/cassava-npy-train-images/train_npy_images'
DIR_INPUT  = '../input/cassava-train-csv'

index_label_map = {
                0: "Cassava Bacterial Blight (CBB)", 
                1: "Cassava Brown Streak Disease (CBSD)",
                2: "Cassava Green Mottle (CGM)", 
                3: "Cassava Mosaic Disease (CMD)", 
                4: "Healthy"
                }

class_names = [value for key,value in index_label_map.items()]

## Helper functions

In [27]:
def find_no_of_trainable_params(model):
    total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_trainable_params

In [28]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

set_seed(CFG.SEED)

In [29]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion matrix', cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt), horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Dataset 

In [30]:
train_df = pd.read_csv(f'{DIR_INPUT}/train.csv')
train_df['npy_image_id'] = train_df['image_id'].str.replace('jpg', 'npy')
train_labels = train_df.iloc[:, 1].values
print(train_df.shape)
print(train_labels.shape)
train_df.head()
folds = StratifiedKFold(n_splits=CFG.N_FOLDS, shuffle=True, random_state=CFG.SEED)

(21397, 3)
(21397,)


In [31]:
class OOFDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.file_names = df['npy_image_id'].values
        self.transforms = transforms
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image = np.load(f'{NPY_FOLDER}/{self.file_names[idx]}')
        if self.transforms:
            augmented = self.transforms(image=image)
            image = augmented['image']
        label = torch.tensor(self.df['label'][idx]).long()
        filename = self.file_names[idx]
        return image, label, filename

## Transforms for Augumentations

In [32]:
def generate_transforms(transform_type):
    heavy_transforms = Compose([
            Resize(height=CFG.SIZE[0], width=CFG.SIZE[1]), #RandomResizedCrop(CFG.size, CFG.size),
            Transpose(p=0.3), VerticalFlip(p=0.3), HorizontalFlip(p=0.3), ShiftScaleRotate(p=0.4),
            RandomBrightnessContrast(p=0.4), 
            IAAAdditiveGaussianNoise(p=0.3),  # sharpen, affine transform
            OneOf([MotionBlur(blur_limit=3), MedianBlur(blur_limit=3), GaussianBlur(blur_limit=3)], p=0.3),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.3),
            CoarseDropout(p=0.4), Cutout(p=0.4),
            Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
            ToTensorV2(p=1.0)])
            # RandomCrop, IAAAdditiveGaussianNoise, RandomResizedCrop(sz,sz),   
            # CLAHE, ImageCompression, MaskDropout, elastictransform, IAAAffine

    basic_transforms = Compose([
            Resize(height=CFG.SIZE[0], width=CFG.SIZE[1]),
            Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0), 
            ToTensorV2(p=1.0)])

    if transform_type == 'basic_transforms':
        return basic_transforms
    
    elif transform_type == 'heavy_transforms':
        return heavy_transforms

## Device

In [33]:
## Device as cpu or tpu
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device('cpu')
print(device)

cuda:0


## Model class

In [34]:
class seResNext50Classifier(nn.Module):
    def __init__(self, model_arch, pretrained=False):
        super(seResNext50Classifier, self).__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.fc.in_features
        self.model.fc = nn.Linear(n_features, CFG.NUM_CLASSES)

    def forward(self, x):
        x = self.model(x)
        return x
    
    
class ViTBase16Classifier(nn.Module):
    def __init__(self, model_arch, pretrained=False):
        super(ViTBase16Classifier, self).__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        self.model.head = nn.Linear(self.model.head.in_features, CFG.NUM_CLASSES)
        
    def forward(self, x):
        x = self.model(x)
        return x

    
class EfficientnetClassifier(nn.Module):
    def __init__(self, model_arch, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, CFG.NUM_CLASSES)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [35]:
model_list = []
#os.listdir(model_path)
print(f'Loading {CFG.WGT_PATH}/{CFG.WGT_MODEL} model weights')
    
if CFG.MODEL_ARCH   == 'resnext50_32x4d':
    model = seResNext50Classifier(CFG.MODEL_ARCH, pretrained=False)
elif CFG.MODEL_ARCH == 'vit_base_patch16_384':
    model = ViTBase16Classifier(CFG.MODEL_ARCH, pretrained=False)
else:
    model = EfficientnetClassifier(CFG.MODEL_ARCH, pretrained=False)
model.to(device);

Loading ../input/cassava-final-submission-weight-files/resnext50_baseline_v2 model weights


In [36]:
cm_list = []
for i_fold in range(CFG.N_FOLDS):
    file = f'{CFG.WGT_PATH}/{CFG.WGT_MODEL}_fold{i_fold}.pth'
    #print(file)
    checkpoint = torch.load(file)
    #print(checkpoint.keys())
    labels = checkpoint['val_labels']
    preds = checkpoint['val_preds']
    cm = confusion_matrix(labels, preds, normalize='true')
    cm_list.append(cm)

    #model.load_state_dict(checkpoint['model'])
    #print(f'Model loaded for {file}')
    #model_list.append(model)
#print(f'There are {len(model_list)} models in total')

print(f'{CFG.WGT_MODEL} has confusion matrix:')
cm_list = np.array(cm_list)
print(cm_list.mean(axis=0))

resnext50_baseline_v2 has confusion matrix:
[[0.64864499 0.07078172 0.02022576 0.04414662 0.2162009 ]
 [0.05345391 0.81360563 0.01919271 0.04157027 0.07217747]
 [0.01466453 0.02389148 0.78247151 0.1160987  0.06287378]
 [0.00182391 0.00440805 0.01063968 0.97545261 0.00767575]
 [0.07100474 0.05199217 0.0438549  0.07140212 0.76174607]]


In [37]:
print(cm_list.std(axis=0))

[[0.06042586 0.02766936 0.00743773 0.01069293 0.04360752]
 [0.00748864 0.03489862 0.00826721 0.01167325 0.0151094 ]
 [0.0067445  0.00489783 0.01302603 0.00881194 0.01348053]
 [0.00087967 0.00051619 0.00231702 0.0046121  0.00273305]
 [0.01226164 0.00867183 0.00636491 0.00889123 0.01880469]]


In [ ]:
#plt.figure(figsize=(5,5))
#plot_confusion_matrix(cm_list[0], class_names, normalize=True)
#plt.figure(figsize=(5,5))
#plot_confusion_matrix(cm_list[1], class_names, normalize=True)
#plt.figure(figsize=(5,5))
#plot_confusion_matrix(cm_list[2], class_names, normalize=True)
#plt.figure(figsize=(5,5))
#plot_confusion_matrix(cm_list[3], class_names, normalize=True)
#plt.figure(figsize=(5,5))
#plot_confusion_matrix(cm_list[4], class_names, normalize=True)

In [ ]:
def generate_OOF_results(model,dataloader, i_fold):
    fold_results = {}
    fold_preds   = []
    fold_labels  = []
    fold_files   = []
    iterator = iter(dataloader)
    progress_bar = tqdm(range(len(dataloader)))
    
    model.eval()
    with torch.no_grad():        
        for idx in progress_bar:
            try:
                images, labels, filename = next(iterator)
            except StopIteration:
                iterator = iter(dataloader)
                images, labels, filename = next(iterator)
            images, labels = images.to(device), labels.to(device)

            # predictions
            class_probs = F.softmax(model(images), -1)
            fold_preds.append(class_probs.data.cpu().numpy())
            fold_labels.append(labels.data.cpu().numpy())
            fold_files.append(filename)

    fold_preds = np.concatenate(fold_preds, axis=0)
    fold_labels = np.concatenate(fold_labels, axis=0)
    fold_files = np.concatenate(fold_files, axis=0)
    
    np.save(f'{CFG.WGT_MODEL}_fold{i_fold}_preds.npy', fold_preds)
    np.save(f'{CFG.WGT_MODEL}_fold{i_fold}_labels.npy', fold_labels)
    np.save(f'{CFG.WGT_MODEL}_fold{i_fold}_files.npy', fold_files)

In [ ]:
for i_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df, train_labels)):
    print("Fold {}/{}".format(i_fold + 1, CFG.N_FOLDS))    
    # create fold data
    fold_data = train_df.iloc[valid_idx].reset_index()
    fold_dataset = OOFDataset(fold_data, transforms=generate_transforms('basic_transforms'))
    fold_dataloader = DataLoader(fold_dataset, batch_size= CFG.VAL_BATCH_SIZE, shuffle=False,
                          num_workers=CFG.NUM_WORKERS, pin_memory=False, drop_last=False)
    
    generate_OOF_results(model_list[i_fold], fold_dataloader, i_fold)


In [ ]:
model_preds = []
model_labels = []
model_files = []
model_fold = []

for i_fold in range(5):
    preds  = np.load(f'{CFG.WGT_MODEL}_fold{i_fold}_preds.npy')
    labels = np.load(f'{CFG.WGT_MODEL}_fold{i_fold}_labels.npy')
    files  = np.load(f'{CFG.WGT_MODEL}_fold{i_fold}_files.npy')
    
    model_preds.append(preds)
    model_labels.append(labels)
    model_files.append(files)
    model_fold.extend([i_fold]*len(preds))
    
    acc_score = accuracy_score(labels, np.argmax(preds,axis=1))
    print(f'{CFG.WGT_MODEL} model fold{i_fold} has accuracy of {acc_score}')

model_preds = np.concatenate(model_preds, axis=0)
model_labels = np.concatenate(model_labels, axis=0)
model_files = np.concatenate(model_files, axis=0)
model_fold = np.array(model_fold)

In [ ]:
model_oof_df = pd.DataFrame()
model_oof_df['fold']   = model_fold
model_oof_df['files']  = model_files
model_oof_df['label']  = model_labels

for i in range(5):
    model_oof_df[f'preds{i}'] = model_preds[:,i]

In [ ]:
model_oof_df.sort_values(by=['files'],inplace=True)
model_oof_df.head()

In [ ]:
train_df.sort_values(by=['image_id'],inplace=True)
train_df.head()

In [ ]:
print(np.array_equal(train_df.label.values, model_oof_df.label.values))
print(np.array_equal(train_df.npy_image_id.values, model_oof_df.files.values))
y_hat = np.argmax(model_oof_df.iloc[:, 3:].values, axis=1)
print(accuracy_score(train_df.label.values, y_hat))

In [ ]:
train_df.to_csv(f'{CFG.WGT_MODEL}_train.csv')
model_oof_df.to_csv(f'{CFG.WGT_MODEL}_OOF.csv')